<a href="https://colab.research.google.com/github/vibhu18116/IR-Project-Group23-PatentPal/blob/main/Summary/BigPatent_Baseline_BetterNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle
data = pickle.load(open("/content/drive/MyDrive/bigPatentData/data_pik", 'rb'))

In [3]:
import pandas as pd
df = pd.DataFrame(data[400:500])

In [ ]:
%%time
%%bash

apt-get install apt-utils dselect dpkg

echo "OSTYPE=$OSTYPE"
if [[ "$OSTYPE" == "cygwin" ]] || [[ "$OSTYPE" == "msys" ]] ; then
    echo "Windows or Windows-like environment detected, script not tested, and may not work."
    echo "Try installing the components mention in the install-[ostype].sh scripts manually."
    echo "Or try running under CGYWIN or git-bash."
    echo "If successfully installed, please contribute back with the solution via a pull request, to https://github.com/neomatrix369/awesome-ai-ml-dl/"
    echo "Please give the file a good name, i.e. install-windows.sh or install-windows.bat depending on what kind of script you end up writing"
    exit 0
elif [[ "$OSTYPE" == "linux-gnu" ]] || [[ "$OSTYPE" == "linux" ]]; then
    TARGET_OS="linux"
else
    TARGET_OS="macos"
fi

if [[ -e ../../library/org/neomatrix369 ]]; then
  echo "Library source found"
  
  cd ../../build
  
  echo "Detected OS: ${TARGET_OS}"
  ./install-${TARGET_OS}.sh || true
else
  if [[ -e awesome-ai-ml-dl/examples/better-nlp/library ]]; then
     echo "Library source found"
  else
     git clone "https://github.com/neomatrix369/awesome-ai-ml-dl"
  fi

  echo "Library source exists"
  cd awesome-ai-ml-dl/examples/better-nlp/build

  echo "Detected OS: ${TARGET_OS}"
  ./install-${TARGET_OS}.sh || true 
fi

In [5]:
!python -m spacy download en_core_web_sm
!python -m spacy link en_core_web_sm en || true

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')

✘ Link 'en' already exists
To overwrite an existing link, use the --force flag



In [ ]:
!pip install textacy

In [7]:
!pip install pytextrank

     |████████████████████████████████| 12.8MB 337kB/s 
     |████████████████████████████████| 1.1MB 48.9MB/s 
     |████████████████████████████████| 9.1MB 47.2MB/s 
     |████████████████████████████████| 460kB 28.3MB/s 
     |████████████████████████████████| 122kB 42.1MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=49f2f979b84771a5c6618f4539d6bc1cd4f8358c649f15f11107aa349d1da227
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
ERROR: textacy 0.10.1 has requirement spacy<3.0.0,>=2.2.0, but you'll have spacy 3.0.5 which is incompatible.
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalli

In [8]:
import sys
sys.path.insert(0, '../../library')
sys.path.insert(0, './awesome-ai-ml-dl/examples/better-nlp/library')

from org.neomatrix369.better_nlp import BetterNLP

import pprint
pp = pprint.PrettyPrinter(indent=4)

This version of Python is 64 bits.
This version of Python is 64 bits.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
This version of Python is 64 bits.
This version of Python is 64 bits.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
This version of Python is 64 bits.


In [10]:
betterNLP = BetterNLP()

In [13]:
pred_summaries = []
c = 0
for i in df['description']:
  summarised_result = betterNLP.summarise(i)
  print(c, end=" ")
  c+=1
  pred_summaries.append(summarised_result['summarised_text']) 

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 

In [17]:
len(pred_summaries)

100

In [14]:
!pip install rouge_score

In [20]:
import random
from rouge_score import rouge_scorer
import numpy as np

def calc_rouge_scores(pred_summaries, gold_summaries, 
                                 keys=['rouge1', 'rouge2', 'rougeL'], use_stemmer=True):
    #Calculate rouge scores
    scorer = rouge_scorer.RougeScorer(keys, use_stemmer= use_stemmer)
    
    n = len(pred_summaries)
    
    scores = [scorer.score(pred_summaries[j], gold_summaries[j]) for 
              j in range(n)] 
    
    dict_scores={}                                                            
    for key in keys:
        dict_scores.update({key: {}})
        
    
    for key in keys:
        
        precision_list = [scores[j][key][0] for j in range(len(scores))]
        recall_list = [scores[j][key][1] for j in range(len(scores))]
        f1_list = [scores[j][key][2] for j in range(len(scores))]

        precision = np.mean(precision_list)
        recall = np.mean(recall_list)
        f1 = np.mean(f1_list)
        
        dict_results = {'recall': recall, 'precision': precision, 'f1': f1}
        
        dict_scores[key] = dict_results
        
    return dict_scores

In [21]:
scores_nlp = calc_rouge_scores(pred_summaries, df['abstract'].tolist())

In [22]:
scores_nlp

{'rouge1': {'f1': 0.3047666031290094,
  'precision': 0.4815971350879893,
  'recall': 0.24548178845442295},
 'rouge2': {'f1': 0.091693001721826,
  'precision': 0.14368605759486788,
  'recall': 0.07374393806805904},
 'rougeL': {'f1': 0.18009624067488889,
  'precision': 0.2906240322281898,
  'recall': 0.14485131490867353}}

In [29]:
pred_summaries_tfidf = []
c = 0
for i in df['description']:
  summarised_result = betterNLP.summarise(i, method="tfidf-ignore-stopwords")
  print(c, end=" ")
  c+=1
  pred_summaries_tfidf.append(summarised_result['summarised_text']) 

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 87 88 89 90 91 92 93 94 95 96 97 98 99 

In [25]:
scores_nlp_tfidf = calc_rouge_scores(pred_summaries_tfidf, df['abstract'].tolist())

In [30]:
scores_nlp_tfidf_stop = calc_rouge_scores(pred_summaries_tfidf, df['abstract'].tolist())

In [31]:
scores_nlp_tfidf_stop

{'rouge1': {'f1': 0.30397328024402964,
  'precision': 0.42680644135870194,
  'recall': 0.2566366500038883},
 'rouge2': {'f1': 0.07562766228058963,
  'precision': 0.10857469633183936,
  'recall': 0.06300195366883611},
 'rougeL': {'f1': 0.16791762084600478,
  'precision': 0.23754228008623712,
  'recall': 0.1414726735818843}}

In [26]:
scores_nlp_tfidf

{'rouge1': {'f1': 0.30397328024402964,
  'precision': 0.42680644135870194,
  'recall': 0.2566366500038883},
 'rouge2': {'f1': 0.07562766228058963,
  'precision': 0.10857469633183936,
  'recall': 0.06300195366883611},
 'rougeL': {'f1': 0.16791762084600478,
  'precision': 0.23754228008623712,
  'recall': 0.1414726735818843}}